In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False
clear_model_results = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import os
import pandas as pd
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df
)
from openavmkit.utilities.settings import (
    load_settings
)
from openavmkit.checkpoint import (
    from_checkpoint,
    read_checkpoint,
    write_checkpoint,
    delete_checkpoints
)
from openavmkit.benchmark import (
    run_models, 
    MultiModelResults
)
from openavmkit.utilities.plotting import (
    plot_histogram_df
)
from openavmkit.land import (
    run_land_analysis
)

%matplotlib inline

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)
settings = load_settings()

In [ ]:
if clear_checkpoints:
    delete_checkpoints("3-model")

# 2. Model

In [ ]:
# load the data
df = read_checkpoint("2-clean-03-out")

In [ ]:
results : MultiModelResults = run_models(
    df,
    load_settings(),
    use_saved_results=(not clear_model_results),
    verbose=verbose
)

# 3. Land Analysis

In [ ]:
run_land_analysis(load_settings(), verbose)